In [ ]:
# Installation of the requirements
#!python -m pip install -r requirements.txt

In [ ]:
'''
(C) Renata Wong 2023

Qiskit code for testing fidelity of randomised classical shadow on the ground state energy of molecules.

Procedure:
1. Choose a variational ansatz with initial parameters selected at random.
2. Generate a set of random basis change operators.
3. Apply the random operators to change bases in the ansatz.
4. Measure the ansatz in the Pauli Z basis and store the results as a shadow.
5. Obtain the expectation value of the molecular Hamiltonian from the shadow.
6. Optimize for minimum Hamiltonian expectation value. 
7. Feed the calculated angles/parameters back to the ansatz.
8. Repeat steps 3-7 till the optimization is completed. 
9. Output the minimized expectation value of the molecular Hamiltonian and the mean-square-root-error. 

Note: Below we perform calculations on the molecular Hamiltonian of H_2.
To perform calculations on other molecules, you will need to specify their geometry, charge and spin 
to replace the values in the driver. 

Note: predicting_quantum_properties module comes from https://github.com/hsinyuan-huang/predicting-quantum-properties
'''

import numpy as np
from collections import Counter
import time
from functools import partial

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import BravyiKitaevMapper, QubitConverter
from qiskit.algorithms.optimizers import SLSQP

from qiskit_aer import QasmSimulator
from qiskit import QuantumCircuit, execute

from qiskit.circuit.library import EfficientSU2

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit.opflow import I, StateFn, CircuitStateFn

from predicting_quantum_properties.data_acquisition_shadow import randomized_classical_shadow
from predicting_quantum_properties.prediction_shadow import estimate_exp




# SPECIFY THE NUMBER OF EXPERIMENTS YOU WANT TO RUN
num_experiments = 1

# SPECIFY THE EXPECTED GROUND STATE ENERGY FOR THE MOLECULE OF INTEREST
EXPECTED_EIGENVALUE = -1.86

# SPECIFY THE GEOMETRY OF THE MOLECULE IN QUESTION
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)


problem = driver.run()
hamiltonian = problem.hamiltonian

# The electronic Hamiltonian of the system
second_q_op = hamiltonian.second_q_op()

# Solving the electronic structure problem = determine the ground state energy of the molecule
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import BravyiKitaevMapper

# The Bravyi-Kitaev repserentation of the Fermionic Hamiltonian
mapper = BravyiKitaevMapper()
bkenc_hamiltonian = mapper.map(second_q_op)

print(bkenc_hamiltonian)

In [ ]:
'''
Format Hamiltonian terms and coefficients as required by the package predicting-quantum-properties
'''

hamiltonian_terms = []
weights = []

for observable in bkenc_hamiltonian:
    
    observable_str = str(observable)
    observable_str_clean = observable_str.strip()  # removes white spaces
    pauli_str_list = observable_str_clean.split('*')
    tuple_list = []
    
    for op_index, pauli_op in enumerate(pauli_str_list[1]):
        if pauli_op == 'I' or pauli_op == 'X' or pauli_op == 'Y' or pauli_op == 'Z':
            tuple_list.append((pauli_op, op_index-1))
    if len(tuple_list) > 0:
        hamiltonian_terms.append(tuple_list)
        weights.append(float(pauli_str_list[0].strip()))

system_size = len(hamiltonian_terms[0])

print(hamiltonian_terms)
print(weights)




'''
Reformatting the Hamiltonian for use in estimate_exp(): removing all entries with Pauli I. 
'''

hamiltonian_terms_XYZ = []

for term in hamiltonian_terms:
    term_XYZ = []
    for pauli in term:
        if pauli[0] != 'I':
            term_XYZ.append(pauli)
    hamiltonian_terms_XYZ.append(term_XYZ)         
    
print(hamiltonian_terms_XYZ)

In [ ]:
'''
Choose a variational ansatz.
Note that for molecules other than H_2 you may need to specify a different number of reps.
'''

reps = 1   
ansatz = EfficientSU2(system_size, su2_gates=['rx', 'ry'], entanglement='circular', reps=reps, skip_final_rotation_layer=True)

    
ansatz.decompose().draw('mpl')

In [ ]:
'''
Define the cost function
'''

    
backend = QasmSimulator(method='statevector', shots=1)

# Create circuit with just the randomised basis change operators
def rand_meas_circuit(pauli_op):
    rand_meas = QuantumCircuit(ansatz.num_qubits)
    for idx, op in enumerate(pauli_op):
        if op == 'X':
            rand_meas.x(idx)
        elif op == 'Y':
            rand_meas.y(idx)
        elif op == 'Z':
            rand_meas.z(idx)
    return rand_meas


def objective_function(operators, params):
    
    # Assign parameters to the ansatz and simulate it
    # Generate circuits to measure random Paulis, one circuit for each Pauli
    
    shadow = []
    for pauli_op in operators:
        
        qc = ansatz.bind_parameters(params)
        qc.compose(rand_meas_circuit(pauli_op))
        qc.measure_all()
        result = execute(qc, backend, shots=1).result()
        counts = result.get_counts()
        
        
        # We perform one single shot => index(1)
        # store the shadow in the form [[(Z,1),(Z,-1)...], [(Y,-1),(X,-1),...]] where inner list = snapshot
        # Because measurement output in Qiskit gives us states and not eigenvalues, we need to convert 0->1 and 1->-1
        
        output_str = list(list(counts.keys())[list(counts.values()).index(1)])
        output = [int(i) for i in output_str]
        eigenvals = [x+1 if x == 0 else x-2 for x in output]
        snapshot = [(op, eigenval) for op, eigenval in zip(pauli_op, eigenvals)]
        shadow.append(snapshot)
        
      
    
    # Now, we want to get the expectation values for the Hamiltonian from the shadow using the function
    # estimate_exp(full_measurement, one_observable)
    # where full_measurement = shadow and one_observable is any term in the Hamiltonian with I occurrences removed
    # cost = the total expectation value of the Hamiltonian in the present run
    
    cost = 0.0
    for term, weight in zip(hamiltonian_terms_XYZ, weights):
        sum_product, match_count = estimate_exp(shadow, term)
        if match_count != 0:
            exp_val = sum_product / match_count
            cost = cost + (weight * exp_val)
            
        
    cost_history.append(cost)
            
    return cost


In [ ]:
'''
Generate a random basis change scheme for the ansatz. 
Number of measurements set to 1000, as required by the experiment in https://arxiv.org/abs/2103.07510.
Only generate the random measurements once and use them in all VQE iterations.
'''

num_measurements = 10 

basis_change_scheme = randomized_classical_shadow(num_measurements, system_size)

tuples = (tuple(pauli) for pauli in basis_change_scheme)
counts = Counter(tuples)

print(counts)



'''
Classical optimisation step
'''

# Counter for the execution time
start_time = time.time()

# Classical optimizer
optimizer = SLSQP(maxiter=500)  

cost_function = partial(objective_function, basis_change_scheme)

# Collect the expectation value from each experiment
expectation_values = []

for _ in range(num_experiments):
    cost_history = []
    params = np.random.rand(ansatz.num_parameters)
    result = optimizer.minimize(fun=cost_function, x0=params)
    expectation_values.append(min(cost_history))     # result.fun doesn't output the lowest value found
    print("GROUND STATE ENERGY FOUND = ", min(cost_history))

    
'''
Calculate the error.
'''
rmse_randomised_cs = np.sqrt(np.sum([(EXPECTED_EIGENVALUE - expectation_values[i])**2 
                                     for i in range(num_experiments)])/num_experiments)
print(f"The average root-mean-squared error for regular classical shadow: {rmse_randomised_cs}")



elapsed_time = time.time() - start_time
print("Execution time = ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
'''
Above we have assumed a particular ground state energyfor the molecule of interest.
Below we corroborate this assumption using a classical minimum eigensolver on our Hamiltonian.
'''

converter = QubitConverter(BravyiKitaevMapper())

numpy_solver = NumPyMinimumEigensolver()   

calc = GroundStateEigensolver(converter, numpy_solver)
res = calc.solve(problem)
print('Electronic ground state energy:\n', res) 

In [ ]:
'''
Reconstructing the experimental results in https://arxiv.org/abs/2103.07510

   GROUND STATE ENERGIES and ERRORS for 10 experiments at 1000 measurements (runtime: ca. 2h15m per experiment):
1. -0.9937592690442375       0.8662407309557626
2. -0.7843307493091094       1.0756692506908907
3. -0.8064150045094017       1.0535849954905983
4. -0.9080709120492695       0.9519290879507306
5. -1.1852387172541334       0.6747612827458667
6. -0.9724661353560656       0.8875338646439345
7. -0.44211888783113645      1.4178811121688637
8. -0.9240849183267326       0.9359150816732675
9. -0.7667485196974478       1.0932514803025524
10.-0.8810239823998636       0.9789760176001365
'''
avg_ground_state_ener = np.sum([-0.9937592690442375, -0.7843307493091094, -0.8064150045094017, -0.9080709120492695, 
                    -1.1852387172541334, -0.9724661353560656, -0.44211888783113645, -0.9240849183267326,
                               -0.7667485196974478, -0.8810239823998636]) / 10
print('Average ground state energy over 10 runs:', avg_ground_state_ener)

'''
Average error over 10 experiments: 
'''
avg_error = np.sum([0.8662407309557626, 1.0756692506908907, 1.0535849954905983, 0.9519290879507306, 0.6747612827458667, 
                   0.8875338646439345, 1.4178811121688637, 0.9359150816732675, 1.0932514803025524, 0.9789760176001365]) / 10

print('Average error over 10 runs:', avg_error)